In [13]:
#pip install psycopg2-binary


In [14]:
import pandas as pd
import re
from urllib.parse import urlparse

In [15]:
# Read the Sales Data file
df_SalesInfo = pd.read_csv('C:\\Users\\pc\\Downloads\\Test\\salesData.csv',  encoding='ISO-8859-1') 
df_MovieTable = pd.read_csv('allMoviesWithUniqueID.csv', sep=None, engine='python', encoding='ISO-8859-1')
df_MovieTable = df_MovieTable.rename(columns={"ID": "Movie_ID"})

# View the columns to check if it works
#print(df1.columns)

#drop all columns except title
df_SalesInfo = df_SalesInfo.drop(columns=['year', 'release_date', 'genre',
       'Unnamed: 8', 'opening_weekend', 'theatre_count', 'avg run per theatre',
       'runtime', 'keywords', 'creative_type', 'url'])

def clean_text(text):
    if pd.isna(text):  # Check if the text is NaN
        return ''  # Return an empty string or handle NaN as needed
    # Convert text to lowercase
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


# Apply the cleaning function to the 'title' column
df_SalesInfo['title'] = df_SalesInfo['title'].apply(clean_text)

df_SalesInfo['normalized_title'] = df_SalesInfo['title'].apply(clean_text)
df_MovieTable['normalized_title'] = df_MovieTable['TITLE'].apply(clean_text)

df_SalesInfo = df_SalesInfo.merge(df_MovieTable[['normalized_title', 'Movie_ID']], 
                                             on='normalized_title', how='left')


df_SalesInfo = df_SalesInfo.drop_duplicates()

print(df_SalesInfo.columns)

print(df_SalesInfo.tail())



Index(['title', 'international_box_office', 'domestic_box_office',
       'worldwide_box_office', 'production_budget', 'normalized_title',
       'Movie_ID'],
      dtype='object')
                             title  international_box_office  \
31594       jokbeoldu sinmun iyagi                   12356.0   
31595             my salinger year                  914119.0   
31596            escort vehicle 36                  240000.0   
31597                      the dry                16987526.0   
31598  posledniy bogatyr koren zla                33396899.0   

       domestic_box_office  worldwide_box_office  production_budget  \
31594                  NaN               12356.0                NaN   
31595              54730.0              968849.0                NaN   
31596                  NaN              240000.0                NaN   
31597             364397.0            17351923.0                NaN   
31598                  NaN            33396899.0                NaN   

       

In [16]:
print(df_SalesInfo['title'].apply(type).value_counts())


title
<class 'str'>    31575
Name: count, dtype: int64


In [17]:
unmatched_Sales = df_SalesInfo[df_SalesInfo['Movie_ID'].isna()]

print("Unmatched rows in df_SalesInfo:")
print(unmatched_Sales[['title', 'normalized_title']])

#merged_df.to_csv('merged.csv', index=False, encoding='ISO-8859-1')

Unmatched rows in df_SalesInfo:
Empty DataFrame
Columns: [title, normalized_title]
Index: []


In [18]:
df_BudgetTable = df_SalesInfo[['Movie_ID', 'production_budget']]
print(df_BudgetTable.head())

   Movie_ID  production_budget
0         1                NaN
1         2         30000000.0
2         3                NaN
3         4                NaN
4         5         11000000.0


In [19]:
#df_Box_Office.to_csv('BoxOfficeWithUniqueID.csv', index=False, encoding='ISO-8859-1')

In [20]:
import psycopg2
from psycopg2 import sql

In [22]:
try:
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(
        dbname='DMGroupAssignment',
        user='postgres',
        password='Okpanachi',
        host='localhost',
        port='5432'
    )

    cursor = conn.cursor()

    # Create the ProductionBudget table
    create_table_query = """
    CREATE TABLE IF NOT EXISTS ProductionBudget (
        Movie_ID INTEGER PRIMARY KEY,
        production_budget DECIMAL(15, 2)
    );
    """
    cursor.execute(create_table_query)
    conn.commit()

    # Insert data into the ProductionBudget table with an upsert mechanism
    insert_query = """
    INSERT INTO ProductionBudget (Movie_ID, production_budget)
    VALUES (%s, %s)
    ON CONFLICT (Movie_ID) DO UPDATE
    SET production_budget = EXCLUDED.production_budget;
    """

    # Loop through the DataFrame and execute the insert query for each row
    for index, row in df_BudgetTable.iterrows():
        cursor.execute(insert_query, (row['Movie_ID'], row['production_budget']))

    # Commit the transaction to save changes to the database
    conn.commit()

except Exception as error:
    print(f"Error: {error}")

finally:
    # Ensure the cursor and connection are closed
    if cursor:
        cursor.close()
    if conn:
        conn.close()
